# Web Mining

Another source of data for analysis comes from web pages that can be scraped. Scraping is the art of extracting data from websites. 

## Web Scraping

### Components of a Website

Websites are created using HTML (Hypertext Markup Language), along with CSS (Cascading Style Sheets) and JavaScript. Here's what these things do ([source](https://blog.hubspot.com/marketing/web-design-html-css-javascript)):
* HTML provides the basic structure of sites, which is enhanced and modified by other technologies like CSS and JavaScript.
* CSS is used to control presentation, formatting, and layout.
* JavaScript is used to control the behavior of different elements.

In web scraping, we are mostly concerned with HTML. HTML contains components known as elements or "HTML tags," which control how content within the tags are displayed in the browser. Types of tags include image tags, paragraph tags, header tags, and div tags. These tags have have attributes (such as "class" and "id" information), which we can reference to extract the data inside. You can view the source code of a web page by right clicking on it in your browser and clicking "Inspect" (Chrome).  

### Try It

Go to the URL: https://www.bikemap.net/en/r/98460/ and inspect the source code on the page. Specifically inspect the description of the route under "About this route." What is the class of the div that contains the route information?

## HTTP Requests

In order to fetch the HTML document for rendering, web browsers issue HTTP requests. We can simulate this action using the `requests` module. For your reference, the link to the `requests` documentation is below. 

https://requests.kennethreitz.org/en/master/ 

In [ ]:
import requests

In [ ]:
bike_url = 'https://www.bikemap.net/en/r/98460/'
# Issue a simple HTTP request to get the webpage text
bike_page = requests.get(bike_url)
# Response code is returned
bike_page

In [ ]:
# Use .text to view the page
# View the first 1000 characters of the HTML document
bike_page.text[:1000]

`requests` can also handle authentication and cookies. See the documentation for more information.

We can also issue HTTP requests via the `urllib.request` module. This is the simplest way to download images. 

https://docs.python.org/3/library/urllib.request.html#module-urllib.request

In [ ]:
import urllib.request

In [ ]:
# We will create an images folder in your current 
# working directory using the os module
import os

In [ ]:
# Create an images folder in the current working directory
my_wd = os.getcwd()
print("My working directory:")
print(my_wd)

In [ ]:
# Path to images folder will be the current working directory + images
img_dir = os.path.join(my_wd,'images')
# If images folder does not exist, create it
if not os.path.exists(img_dir):
    os.makedirs(img_dir)

In [ ]:
# URL of image we want to retrieve
bike_img_url = 'https://media.bikemap.net/routes/98460/staticmaps/98460_1000x260.jpg'

# Use string functions to get the name of the image
bike_img_name = bike_img_url.split("/")[-1]
print(bike_img_name)

In [ ]:
# Use urllib to fetch the image
# Save the image to the images folder in the working directory
# Note that os.path.join is concatenating the path to the
# images directory AND the name of the image e.g. my_image.png
urllib.request.urlretrieve(url=bike_img_url, filename=os.path.join(img_dir,bike_img_name))

### BeautifulSoup

The HTML output we obtained from our HTTP request was not easy to read. We can use the `BeautifulSoup` module to make the HTTP more readable and to search for specific content on the page. We will barely scratch the surface of what you can do with BeautifulSoup. The documentation is below for reference.

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [ ]:
# Often people forget that the package name for BeautifulSoup is bs4
from bs4 import BeautifulSoup

In [ ]:
# Recall that our output from requests was not easy to read
bike_page.text[:1000]

In [ ]:
# First pass the text from the http request to the parser 
bike_page_soup = BeautifulSoup(bike_page.text, 'html.parser')
bike_page_soup

In [ ]:
# soup.find can search for specific kinds of tags (name),
# classes, and ids
# In the case of multiple matches, the first result will be returned
bike_page_soup.find(name="h1", class_="title")

In [ ]:
# .find_all returns all matches as a list
bike_page_soup.find_all(name="div", class_="flag-body")

In [ ]:
# Looking at the output above, it looks like we don't need the 
# flag-body container divs - let's return the item-value spans instead
# .find_all returns all matches in a list
bike_values_list = bike_page_soup.find_all(name="span", class_="item-value")
print(bike_values_list)
print("Number of results: " + str(len(bike_values_list)))

In [ ]:
# Let's extract the text from inside route-distance
# route-distance is the first result
route_distance_str = bike_values_list[0].text
print(route_distance_str)

In [ ]:
# Equivalently, we could have used .find for the id "route-distance"
print(bike_page_soup.find(name="span", id="route-distance").text)

In [ ]:
# Let's use regular expressions to extract the number!
import re

In [ ]:
# Use re.findall to match only the numbers
# Recall: "\d" is a digit - "+" specifies 1 or more times
re.findall('\d+', route_distance_str)

In [ ]:
# Store the first result of the regular expression find as a float
route_distance_num = float(re.findall('\d.', route_distance_str)[0])
print(route_distance_num)

### Practice Problem

At the beginning of class we inspected the element that contains the route description underneathe "About this route." Recall what the div class was (or go look at it again). Extract the text from that particular div using BeautifulSoup.
https://www.bikemap.net/en/r/98460 

## Using Web Scraped Content in Pandas

In [ ]:
from numpy import nan as NA
import numpy as np
import pandas as pd
pd.set_option('max_colwidth',150)

### Putting It All Together - Define a Function to Process a Page

Once you've inspected and extracted all the relevant information you want on the page, you can create a function to process the information from that URL. At a minimum, the function should take in the URL or a part of the URL as an argument. (In our example, we can use the route ID because that is a unique identifier of the URL.)

Store the information in a pandas dataframe.

In [ ]:
# In our example, I will extract the title, description, distance, and image. Note I am using the route ID to construct the URL.
def bike_url_scrape(route_id):
    # Concatenate base URL with route ID to get specific page
    bike_url = 'https://www.bikemap.net/en/r/' + str(route_id) + '/'

    # Request the page HTML, pass to BeautifulSoup parser
    bike_page = requests.get(bike_url)
    bike_page_soup = BeautifulSoup(bike_page.text, 'html.parser')

    # Title
    bike_route_title_raw = bike_page_soup.find(name="h1", class_="title").text
    # Strip leading/trailing whitespace from title
    bike_route_title = bike_route_title_raw.strip()

    # Extract the image - some pages do not have images!
    # First we need to get the image url - it is inside the header carousel
    try:    
        carousel_div = bike_page_soup.find(name="div", id="header-carousel")
        bike_img_url = carousel_div.find('img')['src']
        bike_img_name = str(route_id) + "_" + bike_img_url.split("/")[-1]
        urllib.request.urlretrieve(url=bike_img_url, filename=os.path.join(img_dir,bike_img_name))
    except:
        print("Page does not have a header image.")
    
    # Description
    bike_route_desc = bike_page_soup.find(name="div", class_="route-description").text

    # Distance string
    bike_route_dist_str = bike_page_soup.find(name="span", id="route-distance").text

    # Distance numeric - use regex
    bike_route_dist_num = float(re.findall('\d.', bike_route_dist_str)[0])
    
    # Return result as a dataframe created from a NP array
    temp_row = np.array([[
        route_id, bike_route_title, bike_route_desc, bike_route_dist_str, bike_route_dist_num
        ]])
    temp_df = pd.DataFrame(temp_row, columns = ['route_id', 'bike_route_title', 'bike_route_desc', 'bike_route_dist_str', 'bike_route_dist_num'])
    
    return temp_df

In [ ]:
# Try out some routes: 4994814, 4891998, 3503377, 1903803
bike_scrape_df1 = bike_url_scrape(4994814)
bike_scrape_df1.head()

### Loop Over Pages and DataFrames

In [ ]:
# Import the time module to pause the code in case we are identified as scrapers and blocked!
import time

In [ ]:
# List of route ids we want to scrape
route_ids = [98460, 4994814, 4891998, 3503377, 1903803]

In [ ]:
# Loop over our route IDs and append to the empty dataframe
# Store dataframes in a list
dfs = []
for route_id in route_ids:
    print("Processing Route: " + str(route_id))
    # Scrape route
    bike_scrape_res = bike_url_scrape(route_id)
    # Wait half a second
    time.sleep(0.5)
    # Append df to list
    dfs.append(bike_scrape_res)
    
print("Finished")

In [ ]:
# Use pd.concat to combine the dataframes
bike_scrape_df = pd.concat(dfs)
bike_scrape_df.head()

## Futher Analysis in Pandas

A large set of methods we have already covered can then be applied to the data to answer questions.

### Practice Problems
How many kilometers of routes are available?

How many km across all routes are available?